<a href="https://colab.research.google.com/github/JacopoMangiavacchi/SBERT-ZSC/blob/main/SBERT-Cosine-Similarity-Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
sentences = [
    'This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.'
]

# Test Sentence BERT with Hugginface Transformer 

In [2]:
!pip install transformers

from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
from scipy import spatial

tokenizer = AutoTokenizer.from_pretrained('deepset/sentence_bert')
model = AutoModel.from_pretrained('deepset/sentence_bert')

Some weights of the model checkpoint at deepset/sentence_bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# run inputs through model and mean-pool over the sequence
# dimension to get sequence-level representations
inputs = tokenizer.batch_encode_plus(sentences,
                                     return_tensors='pt',
                                     pad_to_max_length=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [4]:
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
output = model(input_ids, attention_mask=attention_mask)[0]
sentence_rep = output.mean(dim=1)

In [5]:
# find the highest cosine similarities between sentences
print(F.cosine_similarity(sentence_rep[0], sentence_rep[0], dim=0))
print(F.cosine_similarity(sentence_rep[0], sentence_rep[1], dim=0))
print(F.cosine_similarity(sentence_rep[0], sentence_rep[2], dim=0))
print(F.cosine_similarity(sentence_rep[1], sentence_rep[2], dim=0))

tensor(1., grad_fn=<DivBackward0>)
tensor(0.5969, grad_fn=<DivBackward0>)
tensor(-0.1929, grad_fn=<DivBackward0>)
tensor(-0.0781, grad_fn=<DivBackward0>)


In [6]:
embeddings = sentence_rep.detach().numpy()

print(1 - spatial.distance.cosine(embeddings[0], embeddings[0]))
print(1 - spatial.distance.cosine(embeddings[0], embeddings[1]))
print(1 - spatial.distance.cosine(embeddings[0], embeddings[2]))
print(1 - spatial.distance.cosine(embeddings[1], embeddings[2]))

1.0
0.5969038605690002
-0.19288042187690735
-0.07813181728124619


# Test Sentence BERT with Sentence-Transformer

In [7]:
!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 26.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=fefb99ce5c5e495bd4fc7e9dfd20cdcfa8b24e4531ee46bf936009cbcd0661bc
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
print("Max Sequence Length:", model.max_seq_length)

embeddings = model.encode(sentences)

Max Sequence Length: 256


In [9]:
print(1 - spatial.distance.cosine(embeddings[0], embeddings[0]))
print(1 - spatial.distance.cosine(embeddings[0], embeddings[1]))
print(1 - spatial.distance.cosine(embeddings[0], embeddings[2]))
print(1 - spatial.distance.cosine(embeddings[1], embeddings[2]))

1.0
0.5380793213844299
0.11805637180805206
0.10358978062868118
